In [2]:
import torch
import os
import numpy as np
import pandas as pd
import finetune

In [4]:
path_to_models = os.path.join('..', '..', 'data', 'finetuned_models')

datasets = ['FGVC-Aircraft', 'StanfordCars']
model_names = ['resnet', 'densenet', 'vgg']

In [13]:
# Importing all Resnet models in StanfordCars
models = {}

for dataset in datasets:
    path_to_model_dir = os.path.join(path_to_models, dataset, 'resnet')
    for file_name in os.listdir(path_to_model_dir):
        # print(f'{dataset}: {file} | {path_to_model_dir}')
        name = file_name.split('.')[0]
        print(name)
        full_path = os.path.join(path_to_model_dir, file_name)
        model_saved_state = finetune.load_checkpoint(full_path)
        models[name] = model_saved_state
        break


print(models['resnet_FGVC-Aircraft_E10'])



resnet_FGVC-Aircraft_E10
resnet_StanfordCars_E10_occ0
dict_keys(['name', 'epochs', 'model_state_dict', 'optimizer_state_dict', 'occlusion', 'val_acc_history', 'train_acc_history', 'train_loss_history'])
